In [ ]:
import numpy as np
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D

num_classes = 2
resnet_weights_path = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

my_new_model = Sequential()
my_new_model.add(ResNet50(include_top=False, pooling='avg', weights=resnet_weights_path))
my_new_model.add(Dense(num_classes, activation='softmax'))

# Say not to train first layer (ResNet) model. It is already trained
my_new_model.layers[0].trainable = False

In [ ]:
my_new_model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

image_size = 224
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=0.2)


train_generator = data_generator.flow_from_directory(
        '../input/indoorsoutdoors',
        target_size=(image_size, image_size),
        batch_size=15,
        class_mode='categorical',
        subset='training')

validation_generator = data_generator.flow_from_directory(
        '../input/indoorsoutdoors',
        target_size=(image_size, image_size),
        class_mode='categorical',
        subset='validation')

my_new_model.fit_generator(
        train_generator,
        steps_per_epoch=40,
        validation_data=validation_generator,
        validation_steps=1)

In [ ]:
from os.path import join

image_dir = '../input/indoorsoutdoors/'

image_filenames = ['outdoors/1-park.jpg',
                  'indoors/Indoor-Plants-Maui-Kentia-Palm.jpg',
                  'indoors/in.jpg',
                  'outdoors/out.jpg']

img_paths = [join(image_dir, filename) for filename in image_filenames ]

image_size = 224

def read_and_prep_images(img_paths, img_height=image_size, img_width=image_size):
    imgs = [load_img(img_path, target_size=(img_height, img_width)) for img_path in img_paths]
    img_array = np.array([img_to_array(img) for img in imgs])
    output = preprocess_input(img_array)
    
    return(output)

In [ ]:
test_data = read_and_prep_images(img_paths)

preds = my_new_model.predict(test_data)

preds

In [ ]:
from IPython.display import Image, display

for i, img_path in enumerate(img_paths):
    display(Image(img_path))
    if preds[i][0] > preds[i][1]:
        print("Indoors")
    else:
        print("Outdoors")